## Co-moving stars in Kepler

(done in a very hacky way) (but this is probably ok because we're assuming all the stars are far away)

In [1]:
import numpy as np
from astropy.table import Table, unique
from astropy import units as u
import astropy.coordinates as coord
from astropy.time import Time
from astropy.io import fits
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

Load up the table of all Gaia DR2 sources within the Kepler field:

In [2]:
gaia_table_file = '../data/gaia-kepler-dustin.fits'
hdul = fits.open(gaia_table_file)
gaia_src_tbl = Table(hdul[1].data)

In [3]:
#filter = (gaia_src_tbl['phot_g_mean_mag'] >= 18.) & (gaia_src_tbl['parallax'] > 0.01)
#gaia_src_tbl = gaia_src_tbl[filter]

In [4]:
gaia_src_tbl = gaia_src_tbl.to_pandas()

Now load up the Gaia-Kepler cross-match so we can add Kepler IDs to the best-match Gaia sources:

In [5]:
kepler_table_file = '../data/kepler_dr2_1arcsec.fits'
hdul = fits.open(kepler_table_file)
kepler_tbl = Table(hdul[1].data)
gaia_kepler_matches = kepler_tbl['kepid', 'kepler_gaia_ang_dist', 'source_id', 'nconfp', 'nkoi', 'planet?']
gaia_kepler_matches = gaia_kepler_matches.to_pandas()
print(len(gaia_kepler_matches))

195830


Trim off the less-good matches so that there's one unique Gaia source per Kepler target:

In [6]:
gaia_kepler_matches.sort_values(['kepid', 'kepler_gaia_ang_dist'], inplace=True)
gaia_kepler_matches.drop_duplicates('kepid', inplace=True)
print(len(gaia_kepler_matches))

193244


And join the tables:

In [7]:
table = gaia_src_tbl.merge(gaia_kepler_matches, on='source_id', how='left')
len(table)

9376693

In [8]:
test_id = 2105885485289168768
print(gaia_kepler_matches[gaia_kepler_matches['source_id'] == test_id])
print(gaia_src_tbl[gaia_src_tbl['source_id'] == test_id])

          kepid  kepler_gaia_ang_dist            source_id  nconfp  nkoi  \
100000  8151284              0.108671  2105885485289168768       0     0   

       planet?  
100000    none  
                solution_id                   designation  \
979714  1635721458409799680  Gaia DR2 2105885485289168768   

                  source_id  random_index  ref_epoch          ra  ra_error  \
979714  2105885485289168768     486039161     2015.5  285.685012  0.018974   

              dec  dec_error  parallax          ...           e_bp_min_rp_val  \
979714  44.060521   0.020247  0.654823          ...                    0.1898   

        e_bp_min_rp_percentile_lower  e_bp_min_rp_percentile_upper  \
979714                        0.1285                        0.2864   

        flame_flags  radius_val  radius_percentile_lower  \
979714         3503    1.370843                  1.27356   

        radius_percentile_upper  lum_val  lum_percentile_lower  \
979714                 1.658074   2.3351  

Now load up Dustin's pairs:

In [9]:
pairs_file = '../data/matched-pairs-dustin.fits'
hdul = fits.open(pairs_file)
pairs = hdul[0].data

Define some useful functions:

In [11]:
def ppm_check(star1, star2, sigma=5.):
    """
    Returns True if the differences between parallax, pmra, and pmdec are all below 
    the sigma threshold.
    """
    names = ['parallax', 'pmra', 'pmdec']
    for name in names:
        err = np.sqrt(star1[name+'_error']**2 + star2[name+'_error']**2)
        if np.isnan(star1[name]) or np.isnan(star2[name]) or np.isnan(err):
            return False
        if (np.abs(star1[name] - star2[name])/err) >= sigma:
            return False
    return True

In [12]:
def make_x(star):
    """
    returns a vector of x = [parallax, pmra, pmdec]
    """
    names = ['parallax', 'pmra', 'pmdec']
    return np.array([star[n] for n in names])

def make_cov(star):
    """
    returns covariance matrix C corresponding to x
    """
    names = ['parallax', 'pmra', 'pmdec']
    C = np.zeros((3, 3)) # brittle
    for i, name1 in enumerate(names):
        err = star[name1+'_error']
        C[i, i] = err ** 2
        for j, name2 in enumerate(names):
            if j >= i:
                continue
            corr = star["{0}_{1}_corr".format(name2, name1)]
            C[i, j] = corr * np.sqrt(C[i, i] * C[j, j])
            C[j, i] = C[i, j]
    return C

def chisq(star1, star2):
    """
    calculates chisquared for two stars based on their parallax and 2D proper motions
    """
    deltax = make_x(star1) - make_x(star2)
    cplusc = make_cov(star1) + make_cov(star2)
    return np.dot(deltax, np.linalg.solve(cplusc, deltax))

In [13]:
def check_with_primary(m, primary):
    if ppm_check(primary, m):
        return chisq(primary, m)
    else:
        return -1

In [ ]:
chisqs = np.zeros_like(pairs) - 1.

for i,row in tqdm(enumerate(pairs)):
    primary = table.iloc[i]
    matches = table.iloc[row[(row > -1) & (row > i)]] # ignore non-matches and duplicates
    if len(matches) > 0:
        row_of_chisqs = matches.apply(check_with_primary, args=(primary,), axis=1)
        chisqs[i,:len(row_of_chisqs)] = row_of_chisqs.values

57327it [00:47, 1210.51it/s]

In [ ]:
plt.hist(chisqs[(chisqs > 0.) & (chisqs < 10.)], bins=100)
plt.xlabel('$\chi^2$')
plt.ylabel('# Pairs')
plt.yscale('log')